In [ ]:
from utils import get_boot_sector, get_fat_info, get_file_system

In [ ]:
image_file = '/Users/jackbullen/Downloads/subdirs.img'

file_path = '/subdir1/subdir2/foo.txt'
destination_path = './mkfile.cc'

with open(image_file, "rb") as f:
    # Get super block, root, and fat
    os_name, block_size, block_count, fat_start, fat_blocks, root_dir_start, root_dir_blocks = get_boot_sector(f)
    fat = get_fat_info(f, fat_start, fat_blocks, block_size)

    file_system = get_file_system(root_dir_blocks, block_size, fat, root_dir_start)

    path = file_path.split('/')[:-1]
    dir = '/'.join(path)

    file_name = file_path.split('/')[-1]

    print(file_path, file_name)
    with open(destination_path, 'wb') as destination:
        destination.write(file_system[dir]['files'][file_name])